# Explore a SysML Model

**Use case:** "I have a `.sysml` file in Istari and I just want to know what's in it."

This notebook connects to Istari, finds a SysML model, runs the SysGit extraction to produce structured data and diagrams, then displays the results — all without installing a SysML editor.

**What you'll see:**
- The raw SysML text (to understand what we're starting with)
- A structured table of all requirements (IDs, descriptions, target values)
- A structured table of all parts (names, attributes, values)
- Visual diagrams of the requirements hierarchy and parts structure

In [ ]:
# Setup — install SDK and connect to Istari
import sys, os
from pathlib import Path

# Install SDK if needed (e.g. in Colab)
try:
    import istari_digital_client
except ImportError:
    !pip install istari-digital-client python-dotenv -q

# Add repo root to path so we can import istari_client
repo_root = str(Path.cwd().parent.parent)
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from istari_client import get_client

client = get_client()
user = client.get_current_user()
print(f"Connected as: {user.display_name} ({user.email})")

In [ ]:
# Find the model
#
# Replace this with your own model ID, or use the default from the Group3 UAS project.
# You can find model IDs by running: python getting-started/01_explore_systems.py

MODEL_ID = "22bb5f2d-b902-4418-a75c-9e1ae1504c53"  # Group3 UAS SysML model

model = client.get_model(MODEL_ID)
print(f"Model: {model.display_name}")
print(f"  File ID: {model.file.id}")
print(f"  Revisions: {len(model.file.revisions)}")
for rev in model.file.revisions:
    print(f"    - {rev.name} ({rev.size:,} bytes)")
print(f"  Existing artifacts: {len(model.artifacts)}")

In [ ]:
# Peek at the raw SysML text
#
# This is what the file looks like before extraction — SysML v2 syntax with
# requirements, parts, constraints, and satisfy statements all interleaved.

content = model.read_text()
lines = content.splitlines()
print(f"Total: {len(lines)} lines, {len(content):,} characters\n")
print("--- First 40 lines ---")
for i, line in enumerate(lines[:40], 1):
    print(f"{i:4d} | {line}")

In [ ]:
# Run SysGit extraction
#
# This submits a job that parses the SysML file and produces:
#   - output_requirements.json  (structured requirement data)
#   - output_parts.json         (structured parts data)
#   - requirements_hierarchy.png (visual requirements tree)
#   - parts_diagram.png          (visual parts diagram)

from time import sleep
from datetime import datetime
from istari_digital_client import JobStatusName

print("Submitting extraction job...")
job = client.add_job(
    model_id=MODEL_ID,
    function="@istari:extract_sysmlv2",
    tool_name="sysgit",
    tool_version="0.1.8",
    operating_system="Ubuntu 22.04",
    parameters={},
)
print(f"Job: {job.id}")

# Poll until done
while True:
    sleep(5)
    job = client.get_job(job.id)
    ts = datetime.now().strftime("%H:%M:%S")
    status = job.status.name.value
    print(f"  [{ts}] {status}")
    if job.status.name in {JobStatusName.COMPLETED, JobStatusName.FAILED}:
        break

if job.status.name == JobStatusName.COMPLETED:
    print("\nExtraction complete!")
    # Refresh model to see new artifacts
    model = client.get_model(MODEL_ID)
    print(f"Artifacts ({len(model.artifacts)}):")
    for a in model.artifacts:
        rev = a.file.revisions[0] if a.file.revisions else None
        print(f"  - {rev.name if rev else a.name}")
else:
    print("\nExtraction failed!")
    if job.status_history:
        for s in job.status_history:
            print(f"  {s.name}: {getattr(s, 'message', '')}")

In [ ]:
# View requirements
#
# Find the requirements JSON artifact and display it as a readable table.

import json
from IPython.display import HTML, display

# Find the requirements artifact
reqs_artifact = None
for a in model.artifacts:
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev and "requirements" in rev.name and rev.name.endswith(".json"):
        reqs_artifact = a
        break

if reqs_artifact:
    reqs_data = json.loads(reqs_artifact.read_text())
    reqs = reqs_data if isinstance(reqs_data, list) else reqs_data.get("requirements", [])
    
    print(f"Found {len(reqs)} requirements\n")
    
    # Build HTML table
    rows = ""
    for r in reqs:
        req_id = r.get("id", r.get("req_id", "—"))
        name = r.get("name", "—")
        desc = r.get("description", r.get("doc", "—"))
        attrs = r.get("attributes", {})
        attr_str = ", ".join(f"{k}: {v}" for k, v in attrs.items()) if attrs else "—"
        rows += f"<tr><td>{req_id}</td><td><b>{name}</b></td><td>{desc}</td><td>{attr_str}</td></tr>\n"
    
    html = f"""
    <table style="border-collapse: collapse; width: 100%; font-size: 13px;">
    <thead>
        <tr style="background: #f1f5f9;">
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1;">ID</th>
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1;">Name</th>
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1;">Description</th>
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1;">Attributes</th>
        </tr>
    </thead>
    <tbody>{rows}</tbody>
    </table>
    """
    display(HTML(html))
else:
    print("Requirements artifact not found. Run the extraction cell first.")

In [ ]:
# View parts
#
# Find the parts JSON artifact and display as a table.

parts_artifact = None
for a in model.artifacts:
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev and "parts" in rev.name and rev.name.endswith(".json"):
        parts_artifact = a
        break

if parts_artifact:
    parts_data = json.loads(parts_artifact.read_text())
    parts = parts_data if isinstance(parts_data, list) else parts_data.get("parts", [])
    
    print(f"Found {len(parts)} parts\n")
    
    rows = ""
    for p in parts:
        name = p.get("name", "—")
        doc = p.get("doc", p.get("description", "—"))
        attrs = p.get("attributes", {})
        # Show up to 5 key attributes
        attr_items = list(attrs.items())[:5]
        attr_str = "<br>".join(f"{k}: {v}" for k, v in attr_items)
        if len(attrs) > 5:
            attr_str += f"<br><i>+{len(attrs) - 5} more</i>"
        rows += f"<tr><td><b>{name}</b></td><td>{doc}</td><td>{attr_str or '—'}</td></tr>\n"
    
    html = f"""
    <table style="border-collapse: collapse; width: 100%; font-size: 13px;">
    <thead>
        <tr style="background: #f1f5f9;">
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1;">Part</th>
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1;">Description</th>
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1;">Attributes</th>
        </tr>
    </thead>
    <tbody>{rows}</tbody>
    </table>
    """
    display(HTML(html))
else:
    print("Parts artifact not found. Run the extraction cell first.")

In [ ]:
# View diagrams
#
# Display the requirements hierarchy and parts diagram inline.

from IPython.display import Image, display, Markdown

for label, keyword in [("Requirements Hierarchy", "requirements"), ("Parts Diagram", "parts")]:
    png_artifact = None
    for a in model.artifacts:
        rev = a.file.revisions[0] if a.file.revisions else None
        if rev and keyword in rev.name and rev.name.endswith(".png"):
            png_artifact = a
            break
    
    if png_artifact:
        display(Markdown(f"### {label}"))
        img_bytes = png_artifact.read_bytes()
        display(Image(data=img_bytes))
    else:
        print(f"{label}: not found")

## What's Next?

Now that you know what's in the model, you can:

- **Edit a requirement** and re-extract — see [`sysgit/update_and_extract_sysml.py`](../../sysgit/update_and_extract_sysml.py)
- **Run an nTop wing design** using these requirements as targets — see [`ntop/`](../../ntop/)
- **Share the extracted artifacts** with your team — see [`getting-started/03_share_resources.py`](../../getting-started/03_share_resources.py)
- **Upload a new version** of the .sysml file and re-extract to compare — see [`getting-started/02_version_model.py`](../../getting-started/02_version_model.py)